In [1]:
%pip install transformers datasets peft


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
%pip install rake-nltk


In [3]:
%pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.4 MB/s eta 0:00:00


In [4]:
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.3 MB/s eta 0:00:00


In [5]:
from datasets import Dataset, load_dataset

# Load the dataset
dataset = load_dataset("Sharathhebbar24/arxiv-math-instruct-50k")  # Replace with your dataset name

# Convert the 'train' split to a DataFrame
df = dataset['train'].to_pandas()

# Structure the data
def structure_data(row):
    conversation = row['prompt']  # Replace with the actual column name
    user_start_idx = conversation.find('<|user|>') + len('<|user|>')
    assistant_start_idx = conversation.find('<|assistant|>') + len('<|assistant|>')
    question = conversation[user_start_idx:assistant_start_idx].strip()
    answer = conversation[assistant_start_idx:].strip()
    formatted_text = f"Can you answer this question: {question}? The answer is {answer}."
    return {
        "input": formatted_text,  # Store as string
        "output": answer  # Store as string
    }

structured_data = df.apply(structure_data, axis=1).tolist()
dataset_dict = Dataset.from_dict({"data": structured_data})

# Split the dataset: 80% train, 20% val+test
train_val_test_split = dataset_dict.train_test_split(test_size=0.2)
train_data = train_val_test_split['train']

# Split the 20% into 10% validation and 10% test
val_test_split = train_val_test_split['test'].train_test_split(test_size=0.5)
val_data = val_test_split['train']
test_data = val_test_split['test']

# Function to check data integrity
def check_data_integrity(data):
    for idx, item in enumerate(data):
        if not isinstance(item['input'], str):
            print(f"Error in input at index {idx}: {item['input']}")
        if not isinstance(item['output'], str):
            print(f"Error in output at index {idx}: {item['output']}")

check_data_integrity(structured_data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50488 [00:00<?, ? examples/s]

In [6]:
import json

# ... (rest of the code remains the same)

# Save the train data to a JSON file
train_data.to_json("train_data.json")

Creating json from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

58353111

In [7]:
# Load the train data from the JSON file
train_data = Dataset.from_json("train_data.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_data.to_json("/content/drive/MyDrive/checkpointsgpt2")

In [25]:
from google.colab import drive

# Make sure to mount your Google Drive
# drive.mount('/content/drive')

# Save the train_data to a JSON file in the checkpointsgpt2 folder
import json
with open('/content/drive/MyDrive/checkpointsgpt2/train_data.json', 'w') as f:
    json.dump([dict(row) for row in train_data], f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
import tiktoken

# Initialize the tokenizer
tokenizer = tiktoken.get_encoding('gpt2')

# Define the tokenization function
def tokenize_function(examples):
    inputs = [ex['input'] for ex in examples['data']]
    outputs = [ex['output'] for ex in examples['data']]

    # Tokenize inputs and outputs
    def encode(text):
        return tokenizer.encode(text)

    input_ids = [encode(input_text) for input_text in inputs]
    output_ids = [encode(output_text) for output_text in outputs]

    # Handle padding and truncation
    max_length = 580
    pad_token_id = tokenizer.encode('[PAD]')[0] if tokenizer.encode('[PAD]') else tokenizer.eos_token_id

    def pad_and_truncate(ids):
        if len(ids) > max_length:
            return ids[:max_length]
        else:
            return ids + [pad_token_id] * (max_length - len(ids))

    padded_input_ids = [pad_and_truncate(ids) for ids in input_ids]
    padded_output_ids = [pad_and_truncate(ids) for ids in output_ids]

    # Generate attention masks
    attention_masks = [[1] * len(ids) + [0] * (max_length - len(ids)) for ids in padded_input_ids]

    return {
        "input_ids": padded_input_ids,
        "labels": padded_output_ids,
        "attention_mask": attention_masks
    }

# Apply the tokenization function to the dataset
tokenized_train_data = train_data.map(tokenize_function, batched=True)
tokenized_val_data = val_data.map(tokenize_function, batched=True)
tokenized_test_data = test_data.map(tokenize_function, batched=True)

# Convert to PyTorch Dataset
import torch
from torch.utils.data import Dataset as TorchDataset, DataLoader

class CustomDataset(TorchDataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']
        self.labels = tokenized_data['labels']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = CustomDataset(tokenized_train_data)
val_dataset = CustomDataset(tokenized_val_data)
test_dataset = CustomDataset(tokenized_test_data)

# Define DataLoader for batching
def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=10, shuffle=False, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False, collate_fn=collate_fn)

Map:   0%|          | 0/40390 [00:00<?, ? examples/s]

Map:   0%|          | 0/5049 [00:00<?, ? examples/s]

Map:   0%|          | 0/5049 [00:00<?, ? examples/s]

In [9]:

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Step 1: Load the pre-trained GPT-2 model with FP16 precision
model_name_or_path = 'gpt2-medium'
device_map = "auto"  # Automatically distribute model across devices if necessary

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map=device_map,           # Distribute model across devices
    torch_dtype=torch.float16       # Use 16-bit precision (FP16)
)

# # Optional: Load the tokenizer if you need to preprocess text
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

# Step 2: Configure LoRA parameters
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,      # Task type: causal language modeling
    inference_mode=False,              # Set to False for training mode
    r=3,                              # Rank of the low-rank update matrices
    lora_alpha=16,                     # Scaling factor for LoRA
    lora_dropout=0.2,                  # Dropout rate
    fan_in_fan_out=False               # Control how matrices are shared
)

# Step 3: Apply LoRA to the model
model = get_peft_model(model, peft_config)

# Step 4: Print the number of trainable parameters in the model
model.print_trainable_parameters()


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 294,912 || all params: 355,118,080 || trainable%: 0.0830


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [10]:

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import torch

# Define training parameters
lr = 3e-4  # Starting learning rate, could experiment with 1e-4 if needed
num_epochs = 1
warmup_steps = 250 # Adjust based on dataset size

# Ensure optimizer only updates trainable parameters (LoRA and any unfrozen layers)
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

# Calculate total training steps
total_steps = len(train_dataloader) * num_epochs

# Learning rate scheduler setup
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps,
)
# Optionally, add gradient clipping
max_grad_norm = 1.0  # Adjust as needed
torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), max_grad_norm)


tensor(0.)

In [11]:

import os
import torch
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm

# Define checkpointing functions
def save_checkpoint(model, optimizer, epoch, loss, checkpoint_dir):
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch}.pt')
    torch.save({
        'epoch': epoch,
        'lora_state_dict': model.peft_model.state_dict(),  # Save only LoRA adapter state dict
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")

def load_checkpoint(checkpoint_path, model, optimizer, device):
    if os.path.isfile(checkpoint_path):
        print(f"Loading checkpoint '{checkpoint_path}'")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.peft_model.load_state_dict(checkpoint['lora_state_dict'])  # Load LoRA adapter weights
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        loss = checkpoint.get('loss', None)
        print(f"Loaded checkpoint '{checkpoint_path}' (epoch {start_epoch})")
        return start_epoch, loss
    else:
        print(f"No checkpoint found at '{checkpoint_path}', starting from scratch")
        return 0, None

# Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the entire model to the GPU
# Checkpoint Path Definition
checkpoint_dir = '/content/drive/MyDrive/checkpointsgpt2'
os.makedirs(checkpoint_dir, exist_ok=True)
latest_checkpoint = os.path.join(checkpoint_dir, 'checkpoint_latest_epoch_2.pt')

# Loading checkpoint (LoRA only)
start_epoch, _ = load_checkpoint(latest_checkpoint, model, optimizer, device)

No checkpoint found at '/content/drive/MyDrive/checkpointsgpt2/checkpoint_latest_epoch_2.pt', starting from scratch


In [12]:
from tqdm import tqdm  # Import tqdm for progress bar

# Training loop
accumulation_steps = 8
best_val_loss = float('inf')  # Initialize with a very large value

for epoch in range(start_epoch, num_epochs):
    total_loss = 0.0
    model.train()

    # Initialize tqdm with train_dataloader for progress bar
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit=' batches')

    # Training phase
    for step, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()

        # Gradient accumulation handling
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item()

        # Update tqdm progress bar
        progress_bar.set_postfix({'training_loss': total_loss / (step + 1)})

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs}: Average training loss: {avg_train_loss:.4f}")


Epoch 1/1: 100%|██████████| 4039/4039 [58:46<00:00,  1.15 batches/s, training_loss=3.07]

Epoch 1/1: Average training loss: 3.0679


In [27]:
import os
import torch

# Define the path to save the model
model_save_path = '/content/drive/MyDrive/checkpointsgpt2'

# Create the directory if it doesn't exist
if not os.path.exists(model_save_path):
    os.makedirs(model_save_path)

# Save the fine-tuned model
torch.save(model.state_dict(), os.path.join(model_save_path, 'QLORAmathsfinetuned_model.pth'))

print("Model saved to Google Drive!")

Model saved to Google Drive!
